<a href="https://colab.research.google.com/github/essiesalari/Australian-Celebrities-Face-Recognition-VGG16/blob/main/Australian_Celebrities_Face_Recognition_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Australian Celebrities Face Recognition-VGG16**

## Import Libraries

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

## Import datatset

In [2]:
! pip install bing_image_downloader

In [3]:
from bing_image_downloader import downloader

In [4]:
# Query
search_queries=[
    'Nicole Kidman',
    'Hugh Jackman',
    'Mel Gibson',
    'Russell Crowe',
    'Liam Hemsworth',
    'Cathy Freeman',
    'Margot Robbie'
]

dataset_path = 'dataset'

# Imgae downloder function
def donwloader(query, n_sample, dataset_name):
  downloader.download(query, limit=n_sample, output_dir=os.path.join(dataset_path, dataset_name))

for query in search_queries:
  donwloader(query, n_sample=20, dataset_name='train')
  donwloader(query, n_sample=10, dataset_name='val')
  donwloader(query, n_sample=5, dataset_name='test')

[%] Downloading Images to /content/dataset/train/Nicole Kidman


[!!]Indexing page: 1

[%] Indexed 20 Images on Page 1.


[%] Downloading Image #1 from http://images5.fanpop.com/image/photos/31800000/Nicole-Kidman-nicole-kidman-31862066-1440-2200.jpg
[%] File Downloaded !

[%] Downloading Image #2 from http://www.hawtcelebs.com/wp-content/uploads/2018/01/nicole-kidman-at-5th-annual-gold-meets-golden-in-los-angeles-01-06-2018-6.jpg
[%] File Downloaded !

[%] Downloading Image #3 from https://celebmafia.com/wp-content/uploads/2017/09/nicole-kidman-hbo-s-post-emmy-awards-reception-09-17-2017-5.jpg
[%] File Downloaded !

[%] Downloading Image #4 from http://images2.fanpop.com/images/photos/4800000/Nicole-Kidman-nicole-kidman-4896048-1024-768.jpg
[%] File Downloaded !

[%] Downloading Image #5 from http://www.hawtcelebs.com/wp-content/uploads/2011/11/Nicole-Kidman-Arrives-at-45th-Annual-CMA-Awards-8.jpg
[%] File Downloaded !

[%] Downloading Image #6 from http://images2.fanpop.com/image/pho

In [5]:
dataset_path='/content/dataset'
train_path = os.path.join(dataset_path, 'train')
val_path = os.path.join(dataset_path, 'val')
test_path = os.path.join(dataset_path, 'test')

In [6]:
nr_files = nr_train_files = nr_val_files = nr_test_files = 0

for root, dirc, files in os.walk(dataset_path):
  nr_files += len(files)
print("#files: ", nr_files)
for root, dirc, files in os.walk(train_path):
  nr_train_files += len(files)
print("#train_files: ", nr_train_files)
for root, dirc, files in os.walk(val_path):
  nr_val_files += len(files)
print("#val_files: ", nr_val_files)
for root, dirc, files in os.walk(test_path):
  nr_test_files += len(files)
print("#test_files: ", nr_test_files)

#files:  241
#train_files:  140
#val_files:  68
#test_files:  33


## Build  a New Model based on VGG16

In [63]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

In [64]:
batch_size = 128
lr = 0.001
epoch = 100

# Define Image data generator
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen =  ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 140 images belonging to 7 classes.
Found 68 images belonging to 7 classes.
Found 33 images belonging to 7 classes.


In [65]:
# Load VGG16
base_model = VGG16(weights='imagenet', include_top=True, input_shape=(224, 224, 3))

# Freeze the layers except the last Conv. layer
for layer in base_model.layers[:-4]:
    layer.trainable = False

# Add the customized classifier
block_5_flatten = Flatten()(base_model.output)
block_5_dense1 = Dense(256, activation='relu')(block_5_flatten)
block_5_norm1 = BatchNormalization()(block_5_dense1)
block_5_dropout1 = Dropout(0.5)(block_5_norm1)

block_5_dense2 = Dense(128, activation='relu')(block_5_dropout1)  # Add another dense layer
block_5_norm2 = BatchNormalization()(block_5_dense2)
block_5_dropout2 = Dropout(0.5)(block_5_norm2)

block_5_dense3 = Dense(64, activation='relu')(block_5_dropout2)  # Add another dense layer
block_5_norm3 = BatchNormalization()(block_5_dense3)
block_5_dropout3 = Dropout(0.5)(block_5_norm3)

predictions = Dense(len(search_queries), activation='softmax')(block_5_dropout3)

# Build a new model
new_model = Model(inputs=base_model.input, outputs=predictions)
new_model.compile(optimizer=Adam(learning_rate=lr), loss='categorical_crossentropy', metrics=['accuracy'])

## Train and Evaluate the New_Model

In [66]:
# Train the new model
steps_per_epoch = round(train_generator.samples / batch_size)
validation_steps = round(val_generator.samples / batch_size)

history = new_model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epoch,
    validation_data=val_generator,
    validation_steps=validation_steps
)

Epoch 1/100
1/1 [==============================] - 10s 10s/step - loss: 2.5213 - accuracy: 0.0833 - val_loss: 1.9457 - val_accuracy: 0.1471
Epoch 2/100
1/1 [==============================] - 6s 6s/step - loss: 2.6921 - accuracy: 0.0833 - val_loss: 1.9454 - val_accuracy: 0.1471
Epoch 3/100
1/1 [==============================] - 3s 3s/step - loss: 3.3072 - accuracy: 0.0000e+00 - val_loss: 1.9456 - val_accuracy: 0.1471
Epoch 4/100
1/1 [==============================] - 6s 6s/step - loss: 2.6632 - accuracy: 0.1406 - val_loss: 1.9456 - val_accuracy: 0.1471
Epoch 5/100
1/1 [==============================] - 5s 5s/step - loss: 2.9972 - accuracy: 0.0833 - val_loss: 1.9456 - val_accuracy: 0.1471
Epoch 6/100
1/1 [==============================] - 5s 5s/step - loss: 2.8862 - accuracy: 0.1328 - val_loss: 1.9456 - val_accuracy: 0.1471
Epoch 7/100
1/1 [==============================] - 4s 4s/step - loss: 2.5201 - accuracy: 0.0833 - val_loss: 1.9455 - val_accuracy: 0.1471
Epoch 8/100
1/1 [===========

KeyboardInterrupt: 

In [67]:
# Evaluate the model
test_loss, test_acc = new_model.evaluate(test_generator)
print('Test accuracy:', test_acc)

1/1 [==============================] - 1s 1s/step - loss: 1.9448 - accuracy: 0.1515
Test accuracy: 0.1515151560306549
